## 0. Requirement

In [1]:
## jalankan ini dahulu sebelum running
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\user\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.parse import quote #pasring space menjadi modulo 20 (%20)

# Web Scraping 
## Case Study: detik.com

In [3]:
## check if there is a response, if it's 200, we are good to go
s = requests.Session()
url = 'https://www.detik.com'
response = s.get(url)
print(response.status_code)

200


In [ ]:
def scrape_detik_byquery(keyword, halaman) -> pd.DataFrame:
    

In [ ]:
def scrape_detik_satu(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Ambil teks tanggal mentah
    tanggal_utuh = soup.find('div', class_='detail__date')
    if tanggal_utuh:
        waktu = tanggal_utuh.get_text(strip=True)

        try:
            # Pisahkan jadi hari, tanggal, dan jam
            hari, sisanya = waktu.split(',', 1)
            sisanya = sisanya.strip()
            parts = sisanya.rsplit(' ', 2)
            tanggal_bersih = parts[0]
            jam = f"{parts[1]} {parts[2]}"
        except Exception:
            hari = np.nan
            tanggal_bersih = np.nan
            jam = np.nan
    else:
        hari = np.nan
        tanggal_bersih = np.nan
        jam = np.nan

    hasil = {
        "judul": soup.find('h1').get_text(strip=True) if soup.find('h1') else np.nan,
        "isi": "\n\n".join(p.get_text(strip=True) for p in soup.find_all('p')),
        "hari": hari,
        "tanggal": tanggal_bersih,
        "jam": jam,
        "kategori": soup.find('div', class_="page__breadcrumb").get_text(strip=True) if soup.find('div', class_="page__breadcrumb") else np.nan,
        "link": url
         
    }

    return hasil

# Fortunately, Detik menyediakan sitemap

In [26]:
# Ambil sitemap (allowed dari detik.com/robots.txt)
url_sitemap = 'https://www.detik.com/sitemap.xml'
response = requests.get(url_sitemap)
soup = BeautifulSoup(response.content, 'xml')  # parsing sebagai XML

# Ambil semua URL sitemap yang disediakan
sitemap_urls = [loc.text for loc in soup.find_all('loc')]
print("Contoh:", sitemap_urls[0])
print("Total sitemap url:", len(sitemap_urls))

Contoh: https://news.detik.com/suara-pembaca/sitemap_web.xml
Total sitemap url: 512


In [13]:
sitemap_berita = sitemap_urls[0] 
resp = requests.get(sitemap_berita)
soup2 = BeautifulSoup(resp.content, 'xml')
artikel_urls = [loc.text.strip() for loc in soup2.find_all('loc')]
print("Total artikel:", len(artikel_urls))

Total artikel: 100


In [17]:
artikel_urls[0]

'https://news.detik.com/suara-pembaca/d-7928775/sulitnya-hapus-data-registrasi-aplikasi-dana'

In [ ]:
def scrape_detik_satu(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Ambil teks tanggal mentah
    tanggal_utuh = soup.find('div', class_='detail__date')
    if tanggal_utuh:
        waktu = tanggal_utuh.get_text(strip=True)

        try:
            # Pisahkan jadi hari, tanggal, dan jam
            hari, sisanya = waktu.split(',', 1)
            sisanya = sisanya.strip()
            parts = sisanya.rsplit(' ', 2)
            tanggal_bersih = parts[0]
            jam = f"{parts[1]} {parts[2]}"
        except Exception:
            hari = np.nan
            tanggal_bersih = np.nan
            jam = np.nan
    else:
        hari = np.nan
        tanggal_bersih = np.nan
        jam = np.nan

    hasil = {
        "judul": soup.find('h1').get_text(strip=True) if soup.find('h1') else np.nan,
        "isi": "\n\n".join(p.get_text(strip=True) for p in soup.find_all('p')),
        "hari": hari,
        "tanggal": tanggal_bersih,
        "jam": jam,
        "kategori": soup.find('div', class_="page__breadcrumb").get_text(strip=True) if soup.find('div', class_="page__breadcrumb") else np.nan,
        "link": url
         
    }

    return hasil

In [ ]:
import pandas as pd

berita = []
for url in artikel_urls[:10]:  # batasi dulu misalnya 10
    try:
        hasil = scrape_detik_satu(url)
        berita.append(hasil)
    except Exception as e:
        print("Gagal:", url, e)

df = pd.DataFrame(berita)
df

,judul,isi,hari,tanggal,jam,kategori,link
0,Sulitnya Hapus Data Registrasi Aplikasi Dana,Saya ingin medaftar akun Dana (PT Espay Debit ...,Jumat,23 Mei 2025,15:32 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7928775...
1,Penawaran investasi PT Rifan Berjangka Tak Ses...,Saya adalah nasabah komoditas berjangka dari P...,Jumat,23 Mei 2025,15:15 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7928735...
2,"Dilindungi Asuransi, Dua Bulan Lebih Perbaikan...","Pada Oktober 2024, saya membeli mobil Ioniq 5 ...",Jumat,23 Mei 2025,14:33 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7928621...
3,Satu Tahun Menunggu Proses Refund Dana Member ...,Kami merupakan salah satu Mitra Member Loket P...,Jumat,16 Mei 2025,11:41 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7916830...
4,"Diklaim Lebih Kuat dari Kaca dan Acrylic, Sola...",Saya membeli Solartuff dari Impact Pratama Gro...,Selasa,13 Mei 2025,22:57 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7912935...
5,Lamanya Proses Instalasi Indosat HiFi,Saya pelanggan Indosat dengan nomor MTX1127979...,Selasa,13 Mei 2025,22:46 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7912923...
6,Janji Diskon Customer Service Inbound Myrepubl...,(SOLVED)\n\n\n\nSaya pelanggan aktif Myrepubli...,Rabu,07 Mei 2025,05:56 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7903019...
7,Akun Shopee Tak Bisa Belanja dengan Voucher,Saya merupakan pelanggan yang aktif belanja on...,Rabu,07 Mei 2025,05:40 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7903016...
8,"Paket Hilang, Ganti Rugi dari JNE Tak Sesuai N...","Sebagai penjual barang di Tokopedia, saya tela...",Rabu,07 Mei 2025,05:32 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7903013...
9,"Barang yang Dijual Dituduh Palsu, Tokopedia Le...",(SOLVED)\n\n\n\nKami adalah penjual di platfor...,Senin,28 Apr 2025,15:24 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7889672...


## lebih lengkap: Scraping seluruh isi sitemap

In [29]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

def get_sitemap_urls(master_sitemap_url: str):
    response = requests.get(master_sitemap_url)
    soup = BeautifulSoup(response.content, 'xml')
    sitemap_urls = [loc.get_text().strip() for loc in soup.find_all('loc')]
    return sitemap_urls

def get_article_urls(sitemap_url: str):
    response = requests.get(sitemap_url)
    soup = BeautifulSoup(response.content, 'xml')
    article_urls = [loc.get_text().strip() for loc in soup.find_all('loc')]
    return article_urls

def scrape_detik(url: str):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Ambil teks tanggal utuh
    tanggal_utuh = soup.find('div', class_='detail__date')
    if tanggal_utuh:
        waktu = tanggal_utuh.get_text(strip=True)
        try:
            hari, sisanya = waktu.split(',', 1)
            sisanya = sisanya.strip()
            parts = sisanya.rsplit(' ', 2)
            tanggal_bersih = parts[0]
            jam = f"{parts[1]} {parts[2]}"
        except Exception:
            hari = np.nan
            tanggal_bersih = np.nan
            jam = np.nan
    else:
        hari = np.nan
        tanggal_bersih = np.nan
        jam = np.nan

    hasil = {
        "judul": soup.find('h1').get_text(strip=True) if soup.find('h1') else np.nan,
        "isi": "\n\n".join(p.get_text(strip=True) for p in soup.find_all('p')),
        "hari": hari,
        "tanggal": tanggal_bersih,
        "jam": jam,
        "kategori": soup.find('div', class_="page__breadcrumb").get_text(strip=True) if soup.find('div', class_="page__breadcrumb") else np.nan,
        "link": url
    }

    return hasil

In [ ]:
all_articles = []
master_sitemap = 'https://www.detik.com/sitemap.xml'

sitemap_urls = get_sitemap_urls(master_sitemap)

# contoh: ambil 2 sitemap dahulu
for sitemap in tqdm(sitemap_urls[:2], desc="Sitemap"):
    try:
        article_urls = get_article_urls(sitemap)

        for url in tqdm(article_urls[:20], desc="Artikel", leave=False):  # batasi 20 artikel dulu
            try:
                article = scrape_detik(url)
                all_articles.append(article)
                time.sleep(1)  # Hindari ban
            except Exception as e:
                print("Gagal scraping artikel:", url, e)

    except Exception as e:
        print("Gagal akses sitemap:", sitemap, e)

# Simpan ke CSV
df = pd.DataFrame(all_articles)
df.to_csv("./files/detik_contoh.csv", index=False, encoding='utf-8-sig')
print("Selesai. Total artikel:", len(df))

Sitemap: 100%|██████████| 3/3 [05:31<00:00, 110.45s/it]

Selesai. Total artikel: 150


In [31]:
df

,judul,isi,hari,tanggal,jam,kategori,link
0,Sulitnya Hapus Data Registrasi Aplikasi Dana,Saya ingin medaftar akun Dana (PT Espay Debit ...,Jumat,23 Mei 2025,15:32 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7928775...
1,Penawaran investasi PT Rifan Berjangka Tak Ses...,Saya adalah nasabah komoditas berjangka dari P...,Jumat,23 Mei 2025,15:15 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7928735...
2,"Dilindungi Asuransi, Dua Bulan Lebih Perbaikan...","Pada Oktober 2024, saya membeli mobil Ioniq 5 ...",Jumat,23 Mei 2025,14:33 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7928621...
3,Satu Tahun Menunggu Proses Refund Dana Member ...,Kami merupakan salah satu Mitra Member Loket P...,Jumat,16 Mei 2025,11:41 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7916830...
4,"Diklaim Lebih Kuat dari Kaca dan Acrylic, Sola...",Saya membeli Solartuff dari Impact Pratama Gro...,Selasa,13 Mei 2025,22:57 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7912935...
...,...,...,...,...,...,...,...
145,Prabowo Didampingi Didit Makan Siang Bersama P...,PresidenPrabowo Subiantodan Perdana Menteri (P...,Minggu,25 Mei 2025,14:03 WIB,detikNewsBerita,https://news.detik.com/berita/d-7931646/prabow...
146,"Ada Long Weekend, Ganjil Genap Jakarta Pekan D...",Pemprov DKI Jakartamenetapkan sistem ganjil-ge...,Minggu,25 Mei 2025,13:58 WIB,detikNewsBerita,https://news.detik.com/berita/d-7931639/ada-lo...
147,"Persiapan Puncak Haji, PPIH Gelar Simulasi Pos...",Petugas Penyelenggara Ibadah Haji (PPIH)mengge...,Minggu,25 Mei 2025,13:57 WIB,detikNewsBerita,https://news.detik.com/berita/d-7931638/persia...
148,Pedoman Upacara Peringatan Harlah Pancasila 1 ...,Hari Lahir Pancasilaakan diperingati pada 1 Ju...,Minggu,25 Mei 2025,13:47 WIB,detikNewsBerita,https://news.detik.com/berita/d-7931618/pedoma...


#### buat fungsi untuk pipelinenya

In [ ]:
def scrape_all_from_sitemap(master_sitemap_url: str, max_sitemap=None, max_articles=None, output_csv: str="detik_semua_artikel.csv"):
    """
    Robot scraping semua artikel dari sitemap utama Detik.

    Args:
        master_sitemap_url (str): URL sitemap utama.
        max_sitemap (int, optional): Batasi jumlah sitemap yang di-scrape. None artinya semua.
        max_articles (int, optional): Batasi jumlah artikel per sitemap. None artinya semua.
        output_csv (str): Nama file output CSV.

    Returns:
        pd.DataFrame: DataFrame berisi hasil scraping artikel.
    """
    all_articles = []

    print("Mengambil daftar sitemap anak...")
    sitemap_urls = get_sitemap_urls(master_sitemap_url)

    if max_sitemap:
        sitemap_urls = sitemap_urls[:max_sitemap]

    for sitemap in tqdm(sitemap_urls, desc="Sitemap"):
        try:
            article_urls = get_article_urls(sitemap)
            if max_articles:
                article_urls = article_urls[:max_articles]

            for url in tqdm(article_urls, desc="Artikel", leave=False):
                try:
                    article = scrape_detik(url)
                    all_articles.append(article)
                    time.sleep(0.5)  # Hindari ban
                except Exception as e:
                    print("Gagal scraping artikel:", url, e)

        except Exception as e:
            print("Gagal akses sitemap:", sitemap, e)

    # Simpan ke CSV
    df = pd.DataFrame(all_articles)
    df.to_csv(output_csv, index=False, encoding='utf-8-sig')
    print(f"Selesai. Total artikel: {len(df)}. Hasil disimpan di {output_csv}")
    return df

In [37]:
df = scrape_all_from_sitemap('https://www.detik.com/sitemap.xml', max_sitemap=10, max_articles=100)
df.info()

Mengambil daftar sitemap anak...


Sitemap: 100%|██████████| 10/10 [26:37<00:00, 159.80s/it]

Selesai. Total artikel: 1000. Hasil disimpan di detik_semua_artikel.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   judul     1000 non-null   object
 1   isi       1000 non-null   object
 2   hari      1000 non-null   object
 3   tanggal   1000 non-null   object
 4   jam       1000 non-null   object
 5   kategori  1000 non-null   object
 6   link      1000 non-null   object
dtypes: object(7)
memory usage: 54.8+ KB


In [38]:
df

,judul,isi,hari,tanggal,jam,kategori,link
0,Sulitnya Hapus Data Registrasi Aplikasi Dana,Saya ingin medaftar akun Dana (PT Espay Debit ...,Jumat,23 Mei 2025,15:32 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7928775...
1,Penawaran investasi PT Rifan Berjangka Tak Ses...,Saya adalah nasabah komoditas berjangka dari P...,Jumat,23 Mei 2025,15:15 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7928735...
2,"Dilindungi Asuransi, Dua Bulan Lebih Perbaikan...","Pada Oktober 2024, saya membeli mobil Ioniq 5 ...",Jumat,23 Mei 2025,14:33 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7928621...
3,Satu Tahun Menunggu Proses Refund Dana Member ...,Kami merupakan salah satu Mitra Member Loket P...,Jumat,16 Mei 2025,11:41 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7916830...
4,"Diklaim Lebih Kuat dari Kaca dan Acrylic, Sola...",Saya membeli Solartuff dari Impact Pratama Gro...,Selasa,13 Mei 2025,22:57 WIB,detikNewsSuara Pembaca,https://news.detik.com/suara-pembaca/d-7912935...
...,...,...,...,...,...,...,...
995,Jumlah Penularan Kasus HMPV Terus Bertambah di...,Beijing mengakui adanya lonjakan kasus human m...,Rabu,08 Jan 2025,18:00 WIB,detikNewsAbc Australia,https://news.detik.com/abc-australia/d-7723523...
996,Dunia Hari Ini: Etihad Batal Lepas Landas di M...,"Anda sedang membaca rangkuman Dunia Hari Ini, ...",Senin,06 Jan 2025,14:57 WIB,detikNewsAbc Australia,https://news.detik.com/abc-australia/d-7719707...
997,Dunia Hari Ini: Pemerintah Korsel Perintahkan ...,Anda sedang membaca rangkuman sejumlah berita ...,Selasa,31 Des 2024,18:20 WIB,detikNewsAbc Australia,https://news.detik.com/abc-australia/d-7711923...
998,Sejumlah Berita dari Indonesia yang Menarik Pe...,Kami mengumpulkan berita-berita soal Indonesia...,Selasa,31 Des 2024,13:20 WIB,detikNewsAbc Australia,https://news.detik.com/abc-australia/d-7711200...


### dengan query
```https://www.detik.com/search/searchall?query=makan%20bergizi%20gratis&page=5&result_type=relevansi``` -> pakai modulo 20

```https://www.detik.com/search/searchall?query={kata_kunci}&page={halaman}&result_type=relevansi```

In [4]:
from urllib.parse import quote, unquote, quote_plus

text = "This is a string with spaces"

encoded_text = quote(text)
print(f"Encoded with quote(): {encoded_text}")

encoded_text_plus = quote_plus(text)
print(f"Encoded with quote_plus(): {encoded_text_plus}")

decoded_text = unquote(encoded_text)
print(f"Decoded text: {decoded_text}")

Encoded with quote(): This%20is%20a%20string%20with%20spaces
Encoded with quote_plus(): This+is+a+string+with+spaces
Decoded text: This is a string with spaces


In [ ]:
def 